In [1]:
import torch
import gc
import os
import ast
import copy
import torch.nn as nn
import torch.optim as optim
print(torch.__version__)
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm
from datetime import datetime
from scipy.stats import pearsonr, zscore
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.fft as fft
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize
from skopt.utils import use_named_args
from torch.cuda.amp import autocast
from torch.utils.data import Sampler
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.data._utils.collate import default_collate
from math import ceil
def apply_hilbert_torch(x, envelope=False, do_log=False, compute_val='power', data_srate=250):
    def hilbert_torch(x):
        N = x.size(-1)
        Xf = fft.fft(x, dim=-1)
        h = torch.zeros(N, dtype=torch.complex64, device=x.device)
        if N % 2 == 0:
            h[0] = h[N // 2] = 1
            h[1:N // 2] = 2
        else:
            h[0] = 1
            h[1:(N + 1) // 2] = 2
        return fft.ifft(Xf * h, dim=-1)
    def angle_custom(z):
        return torch.atan2(z.imag, z.real)
    def unwrap(p, discont=np.pi):
        dp = p[..., 1:] - p[..., :-1]
        ddp = torch.remainder(dp + np.pi, 2 * np.pi) - np.pi
        ddp[torch.abs(dp) < discont] = 0
        p_unwrapped = p.clone()
        p_unwrapped[..., 1:] = p[..., 0][..., None] + torch.cumsum(dp + ddp, dim=-1)
        return p_unwrapped
    def diff(x):
        return x[..., 1:] - x[..., :-1]
    n_x = x.size(-1)
    hilb_sig = hilbert_torch(x)
    
    if compute_val == 'power':
        out = torch.abs(hilb_sig)
        if do_log:
            out = torch.log1p(out)
    elif compute_val == 'phase':
        out = unwrap(angle_custom(hilb_sig))
    elif compute_val == 'freqslide':
        ang = angle_custom(hilb_sig)
        ang = data_srate * diff(unwrap(ang)) / (2 * np.pi)
        out = torch.nn.functional.pad(ang, (0, 1), mode='constant')
        # TO DO: apply median filter (use torch.median or a custom implementation)
    return out

# get gpu device
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

2.8.0+cu126


In [2]:
class CtxNet(nn.Module):
    def __init__(self, Chans, Samples=375, dropoutRate=0.65, kernLength=64, F1=4, 
                 D=2, F2=8, norm_rate=0.25, kernLength_sep=16,
                 do_log=False, data_srate=500, base_split=4):
        super(CtxNet, self).__init__()
        self.do_log = do_log
        self.data_srate = data_srate

        self.block1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernLength), padding='same', bias=False),
            nn.BatchNorm2d(F1),
            nn.Conv2d(F1, F1*D, (Chans, 1), groups=F1, bias=False, padding='same'),
            nn.BatchNorm2d(F1*D),
            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(dropoutRate)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(F1*D, F2, (1, kernLength_sep), bias=False, padding='same'),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(dropoutRate)
        )

        self.flatten = nn.Flatten()

        self.dense = nn.Sequential(
            nn.Linear(self.calculate_flatten_size(Chans, Samples, F2), 64),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.Dropout(dropoutRate)
        )

        self.output = nn.Linear(64, 1)

    def calculate_flatten_size(self, Chans, Samples, F2):
        with torch.no_grad():
            x = torch.randn(1, 1, Chans, Samples)
            x = self.block1(x)
            x = self.block2(x)
            return x.numel()

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.block1(x)
        x = self.apply_hilbert(x)
        x = self.block2(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.output(x)
        return x

    def apply_hilbert(self, x):
        return apply_hilbert_torch(x, do_log=self.do_log, compute_val='power', data_srate=self.data_srate)

    def calculate_flatten_size(self, Chans, Samples, F2):
        with torch.no_grad():
            x = torch.randn(1, 1, Chans, Samples)
            x = self.block1(x)
            x = self.block2(x)
            return x.numel()

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.block1(x)
        x = self.apply_hilbert(x)
        x = self.block2(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.output(x)
        return x

    def apply_hilbert(self, x):
        return apply_hilbert_torch(x, do_log=self.do_log, compute_val='power', data_srate=self.data_srate)

def create_model(Chans, Samples=375, dropoutRate=0.65, kernLength=64, F1=4, D=2, F2=8):
    model = CtxNet(Chans=Chans, Samples=Samples, dropoutRate=dropoutRate, 
                  kernLength=kernLength, F1=F1, D=D, F2=F2)
    return model


def positive_correlation_loss(y_true, y_pred, epsilon=1e-8):
    mx = torch.mean(y_pred)
    my = torch.mean(y_true)
    xm, ym = y_pred - mx, y_true - my
    # Check if we have enough elements to calculate std
    if xm.numel() > 1 and ym.numel() > 1:
        r_num = torch.mean(xm * ym)
        r_den = torch.std(xm) * torch.std(ym)
        r = r_num / (r_den + epsilon)
    else:
        # If we don't have enough elements, return a default loss
        return torch.tensor(1.0, device=y_true.device)
    
    loss = torch.where(r >= 0, 1 - r, 2 - r)
    return loss

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience=7):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    best_model = None
    best_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model = copy.deepcopy(model)

        current_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_loss)
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            break

    return best_model

def objective(params, X, y, n_splits=4):
    kf = KFold(n_splits=n_splits, shuffle=False)
    scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        train_dataset = CustomDataset(X_train, y_train)
        val_dataset = CustomDataset(X_val, y_val)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

        model_params = {
            'Chans': X.shape[1],
            'dropoutRate': params['dropoutRate'],
            'kernLength': params['kernLength'],
            'F1': params['F1'],
            'D': params['D'],
            'F2': params['F2']
        }
        model = create_model(**model_params)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'], 
                                     weight_decay=params['weight_decay'])
        criterion = positive_correlation_loss
        
        trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, 
                                    num_epochs=params['max_epochs'], patience=params['patience'])

        trained_model.eval()
        device = next(trained_model.parameters()).device
        with torch.no_grad():
            y_val_pred = []
            for batch in val_loader:
                inputs, _ = batch
                inputs = inputs.to(device)
                outputs = trained_model(inputs)
                y_val_pred.extend(outputs.cpu().numpy())
            y_val_pred = np.array(y_val_pred).squeeze()
        
        score = positive_correlation_loss(torch.from_numpy(y_val), torch.from_numpy(y_val_pred)).numpy().item()
        scores.append(score)

        # Clear memory
        del model, trained_model, optimizer, train_loader, val_loader
        torch.cuda.empty_cache()
        gc.collect()

    return np.mean(scores)

In [8]:
space = [
    Integer(32, 128, name='kernLength'),
    Integer(2, 8, name='F1'),
    Integer(1, 8, name='D'),
    Integer(4, 16, name='F2'),
    Real(0.3, 0.8, name='dropoutRate'),
    Categorical([1e-3], name='learning_rate'),  # Fixed at 1e-3
    Real(1e-6, 1e-3, "log-uniform", name='weight_decay'),
    Integer(3, 7, name='patience'),
    Categorical([100], name='max_epochs')  # Fixed at 100
]
f_folder = r'C:\Users\admin\1代码 通过ecog数据预测stn的beta\data_upload\demo dataset'
batch_size = 16384
name = 'ctxnet_demo'
y_name = name+'_stnbeta.npy'
y_all = np.load(f'{f_folder}\{y_name}')
x_name = name+'_ecog.npy'
x_all = np.load(f'{f_folder}\{x_name}')
x_all = x_all.reshape((x_all.shape[0], x_all.shape[1], x_all.shape[2]))  

In [9]:
y_all.shape

(14721, 2)

In [10]:
x_all.shape

(14721, 4, 375)

In [11]:
lowbeta = y_all[:,0].squeeze()
highbeta = y_all[:,1].squeeze()  
data = x_all

In [ ]:
# Main loop
pearsonr_value = []
target_list = []
sub_list = []
file_list = []
run_list = []

for beta in ['lowbeta', 'highbeta']:
    targets = locals()[beta]

    X_train_val, X_test, y_train_val, y_test = train_test_split(data, targets, shuffle=False,
                                                                test_size=0.3, random_state=42)

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_train_val_reshaped = X_train_val.reshape(-1, X_train_val.shape[-1])
    X_train_val_normalized = scaler_X.fit_transform(X_train_val_reshaped)
    X_train_val_normalized = X_train_val_normalized.reshape(X_train_val.shape)

    y_train_val_normalized = scaler_y.fit_transform(y_train_val.reshape(-1, 1)).flatten()

    res_gp = gp_minimize(lambda params: objective(dict(zip([dim.name for dim in space], params)), 
                                                  X_train_val_normalized, y_train_val_normalized),
                         space, n_calls=10, random_state=42)

    best_params = dict(zip([dim.name for dim in space], res_gp.x))

    # Train the final model with the best parameters
    model_params = {
        'Chans': data.shape[1],
        'dropoutRate': best_params['dropoutRate'],
        'kernLength': best_params['kernLength'],
        'F1': best_params['F1'],
        'D': best_params['D'],
        'F2': best_params['F2']
    }
    best_model = create_model(**model_params)
    
    optimizer = torch.optim.Adam(best_model.parameters(), lr=best_params['learning_rate'], 
                                 weight_decay=best_params['weight_decay'])
    criterion = positive_correlation_loss
    
    train_dataset = CustomDataset(X_train_val_normalized, y_train_val_normalized)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    
    best_model = train_model(best_model, train_loader, train_loader, criterion, optimizer, 
                             num_epochs=best_params['max_epochs'], patience=best_params['patience'])
    
    best_model.eval()
    device = next(best_model.parameters()).device
    with torch.no_grad():
        # Predict for y_test
        X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
        X_test_normalized = scaler_X.transform(X_test_reshaped)
        X_test_normalized = X_test_normalized.reshape(X_test.shape)
        y_pred_test_normalized = []
        for batch in DataLoader(CustomDataset(X_test_normalized, np.zeros(len(X_test_normalized))), 
                                batch_size=batch_size):
            inputs, _ = batch
            inputs = inputs.to(device)
            outputs = best_model(inputs)
            y_pred_test_normalized.extend(outputs.cpu().numpy())
        y_pred_test_normalized = np.array(y_pred_test_normalized)
        y_pred_test = scaler_y.inverse_transform(y_pred_test_normalized.reshape(-1, 1)).flatten()

        base_filename = os.path.splitext(y_name)[0]
        y_test_combined = np.column_stack((y_test, y_pred_test))

        # Predict for full dataset (y_all)
        X_all_reshaped = x_all.reshape(-1, x_all.shape[-1])
        X_all_normalized = scaler_X.transform(X_all_reshaped)
        X_all_normalized = X_all_normalized.reshape(x_all.shape)
        y_pred_all_normalized = []
        for batch in DataLoader(CustomDataset(X_all_normalized, np.zeros(len(X_all_normalized))), 
                                batch_size=batch_size):
            inputs, _ = batch
            inputs = inputs.to(device)
            outputs = best_model(inputs)
            y_pred_all_normalized.extend(outputs.cpu().numpy())
        y_pred_all_normalized = np.array(y_pred_all_normalized)
        y_pred_all = scaler_y.inverse_transform(y_pred_all_normalized.reshape(-1, 1)).flatten()
        y_all_combined = np.column_stack((targets, y_pred_all))

    pearsonr_value.append(pearsonr(y_pred_test, y_test)[0])
    target_list.append(beta)
    run_list.append('test')
    sub_list.append('sub-demo')
    file_list.append(name)

    # Clear memory
    del best_model, optimizer, train_loader
    torch.cuda.empty_cache()
    gc.collect()

results = pd.DataFrame({
    'value': pearsonr_value, 
    'predict': target_list, 
    'run': run_list, 
    'sub': sub_list,
    'f_name': file_list
})
torch.cuda.empty_cache()
results